# Factor Research Notebook
This notebook demonstrates a complete end-to-end factor research workflow using the Quant Research Platform.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from backtester.core.portfolio import Portfolio
from backtester.strategy.base_strategy import BaseStrategy
from backtester.analytics.performance import PerformanceAnalyzer
from backtester.analytics.statistics import StatisticalValidator
from backtester.optimization.bayesian import BayesianOptimizer
import warnings
warnings.filterwarnings('ignore')

## 1. Generate Sample Factor Data

In [ ]:
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', periods=200)
factor_signal = np.random.randn(200)  # simulated factor
price = 100 + np.cumsum(np.random.randn(200))  # simulated price
data = pd.DataFrame({'price': price, 'factor': factor_signal}, index=dates)
data.head()

## 2. Define a Simple Factor Strategy

In [ ]:
class SimpleFactorStrategy(BaseStrategy):
    def __init__(self, lookback=5):
        self.lookback = lookback
        self.position = 0
    def on_event(self, data_row):
        signal = np.sign(data_row['factor'])
        self.position = signal
        return self.position
    def backtest(self, df):
        returns = df['price'].pct_change().fillna(0)
        positions = df['factor'].apply(lambda x: np.sign(x))
        equity = (1 + returns * positions.shift(1).fillna(0)).cumprod()
        return equity

## 3. Backtest Strategy

In [ ]:
strategy = SimpleFactorStrategy(lookback=5)
equity = strategy.backtest(data)
equity.plot(title='Equity Curve')

## 4. Performance Analytics

In [ ]:
trades = pd.DataFrame({'quantity':[1]*len(data)})
analyzer = PerformanceAnalyzer(equity, trades)
report = analyzer.compute()
report

## 5. Bootstrap & Monte Carlo Validation

In [ ]:
validator = StatisticalValidator(equity.pct_change().dropna())
bootstrap_report = validator.bootstrap_sharpe(n_bootstrap=500)
print('Bootstrap Sharpe Report:', bootstrap_report)

# Monte Carlo Equity Paths
mc_paths = validator.monte_carlo_equity(n_simulations=100)
mc_paths.plot(legend=False, alpha=0.3, title='Monte Carlo Equity Paths')

## 6. Bayesian Hyperparameter Optimization

In [ ]:
def objective(params):
    strat = SimpleFactorStrategy(lookback=int(params['lookback']))
    eq = strat.backtest(data)
    returns = eq.pct_change().dropna()
    sharpe = np.sqrt(252)*returns.mean()/returns.std()
    return sharpe

param_bounds = {'lookback': (1,20)}
optimizer = BayesianOptimizer(param_bounds, objective_fn=objective, n_iter=10)
result = optimizer.optimize()
print('Best Parameters:', result['best_params'])
print('Best Sharpe:', result['best_score'])